In [2]:
from dotenv import load_dotenv
import sys

sys.path.append("../common")
load_dotenv()

True

In [3]:
import os
from langsmith_tracker import set_tracking
from langchain_print import stream_response

# 인스턴스를 생성할 때 필요한 매개변수를 전달합니다.
set_tracking(project_name="03.OutputParser")

Langsmith 추적이 활성화되었습니다. [프로젝트명: 03.OutputParser]


### StructuredOutputParser

`StructuredOutputParser`
- LLM의 응답을 `dict` 형식으로 구성하고, key/value 쌍으로 여러 필드를 반환하고자 할때 사용합니다.    
- Pydantic 을 쓰지 않고 `StructuredOutputParser` 를 써야 할 때가 있습니다.  
GPT 말고도 Local 모델을 쓸때가 있는데, 이때 Pydantic이 적용이 안되는 경우도 있어서 더욱 강력한 `StructuredOutputParser` 를 사용합니다.

`Response Schema`
- 우리가 추출하고자 하는 내용들을 name, description 이라는 값들에 지정해줍니다.

In [4]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

In [8]:
# 사용자 질문에 대한 답변 스키마(양식)
response_schemas = [
    ResponseSchema(name="answer", description="사용자의 질문에 대한 답변"),
    ResponseSchema(
        name="source",
        description="사용자의 질문에 답하기 위해 사용된 `출처`, `웹사이트주소` 이여야 합니다.",
    ),
]

# 응답 스키마(양식)을 기반으로 한 구조화된 출력파서 초기화
output_parser = StructuredOutputParser.from_response_schemas(
    response_schemas=response_schemas
)

print(output_parser.get_format_instructions())

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"answer": string  // 사용자의 질문에 대한 답변
	"source": string  // 사용자의 질문에 답하기 위해 사용된 `출처`, `웹사이트주소` 이여야 합니다.
}
```


In [9]:
format_insturctions = output_parser.get_format_instructions()

prompt = PromptTemplate(
    template="answer the users question as best as possible.\n{format_instructions}\n{question}",
    input_variables=["question"],
    partial_variables={"format_instructions": format_insturctions},
)

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
chain = prompt | llm | output_parser

In [12]:
response = chain.invoke({"question": "비트코인의 발명가는 누구인가요?"})
print(response)

{'answer': '비트코인의 발명가는 사토시 나카모토(Satoshi Nakamoto)라는 가명으로 알려진 개인 또는 그룹입니다. 2008년에 비트코인 백서를 발표하고 2009년에 첫 번째 비트코인 소프트웨어를 출시했습니다.', 'source': 'https://bitcoin.org/bitcoin.pdf'}


`chain.stream` 을 사용합니다.  
OutputParser 는 stream에 제한적입니다. 되도록 invoke를 사용하는 것이 좋습니다. 아직은..

In [14]:
for token in chain.stream({"question": "비트코인의 발명가는 누구인가요?"}):
    print(token)

{'answer': '비트코인의 발명가는 사토시 나카모토(Satoshi Nakamoto)라는 가명으로 알려진 개인 또는 그룹입니다. 2008년에 비트코인 백서를 발표하고 2009년에 첫 번째 비트코인 소프트웨어를 출시했습니다.', 'source': 'https://bitcoin.org/bitcoin.pdf'}
